In [ ]:
!pip install nltk

In [ ]:
import json
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Reading in data

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# file_path_train = "/content/drive/MyDrive/Colab Notebooks/IE4483/data/train.json"


file_path_train = "train.json"

with open(file_path_train, 'r', encoding='utf-8') as file:
    train_data = json.load(file)

In [ ]:
# file_path_test = "/content/drive/MyDrive/Colab Notebooks/IE4483/data/test.json"

file_path_test = "train.json"

with open(file_path_test, 'r', encoding='utf-8') as file:
    test_data = json.load(file)

In [ ]:
# 0: negative sentiment
# 1: positive sentiment

train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

In [ ]:
train_df.head()

,reviews,sentiments
0,I bought this belt for my daughter in-law for ...,1
1,The size was perfect and so was the color. It...,1
2,"Fits and feels good, esp. for doing a swim rac...",1
3,These socks are absolutely the best. I take pi...,1
4,Thank you so much for the speedy delivery they...,1


# Data processing

In [ ]:
#Lowercasing
train_df["reviews"] = train_df["reviews"].astype(str).str.lower()
train_df.head()

test_df["reviews"] = test_df["reviews"].astype(str).str.lower()
test_df.head()

,reviews,sentiments
0,i bought this belt for my daughter in-law for ...,1
1,the size was perfect and so was the color. it...,1
2,"fits and feels good, esp. for doing a swim rac...",1
3,these socks are absolutely the best. i take pi...,1
4,thank you so much for the speedy delivery they...,1


In [ ]:
#Function to remove punctuation
def remove_punc(text):
    for punc in string.punctuation:
        text = text.replace(punc, '')
    return text


train_df["reviews"] = train_df["reviews"].apply(remove_punc)
train_df.head()

test_df["reviews"] = test_df["reviews"].apply(remove_punc)
test_df.head()

,reviews,sentiments
0,i bought this belt for my daughter inlaw for c...,1
1,the size was perfect and so was the color it ...,1
2,fits and feels good esp for doing a swim race ...,1
3,these socks are absolutely the best i take pil...,1
4,thank you so much for the speedy delivery they...,1


In [ ]:
# Remove special characters and numbers from the "reviews" column
train_df['reviews'] = train_df['reviews'].str.replace(r'[^A-Za-z\s]', '', regex=True)

test_df['reviews'] = test_df['reviews'].str.replace(r'[^A-Za-z\s]', '', regex=True)

In [ ]:
# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]
    return ' '.join(filtered_words)

# Apply the remove_stopwords function to the 'reviews' column
train_df['reviews'] = train_df['reviews'].apply(remove_stopwords)

test_df['reviews'] = test_df['reviews'].apply(remove_stopwords)

In [ ]:
findDuplicate = train_df.duplicated()
print(findDuplicate)

0       False
1       False
2       False
3       False
4       False
        ...  
7396    False
7397    False
7398    False
7399    False
7400    False
Length: 7401, dtype: bool


In [ ]:
train_df = train_df.drop_duplicates()
print(train_df)

test_df = test_df.drop_duplicates()
print(test_df)

                                                reviews  sentiments
0            bought belt daughter inlaw christmas loved           1
1               size perfect color looked like web page           1
2     fits feels good esp swim race highly recommend...           1
3     socks absolutely best take pilates classes hot...           1
4     thank much speedy delivery came time rehearsal...           1
...                                                 ...         ...
7396  bought shirts black medium wear daily basis di...           0
7397  first thought scarf might good quality since c...           1
7398  picky comes bras want something support comfor...           1
7399  jacket wind water resistant waterproof soked f...           0
7400  extremely confortable material soft cotton pou...           1

[7178 rows x 2 columns]
                                                reviews  sentiments
0            bought belt daughter inlaw christmas loved           1
1               size pe

In [ ]:
#Save df as csv
train_df.to_csv('processed_train_data.csv', index=False)
test_df.to_csv('processed_test_data.csv', index=False)

In [ ]:
print(train_df)

                                                reviews  sentiments
0            bought belt daughter inlaw christmas loved           1
1               size perfect color looked like web page           1
2     fits feels good esp swim race highly recommend...           1
3     socks absolutely best take pilates classes hot...           1
4     thank much speedy delivery came time rehearsal...           1
...                                                 ...         ...
7396  bought shirts black medium wear daily basis di...           0
7397  first thought scarf might good quality since c...           1
7398  picky comes bras want something support comfor...           1
7399  jacket wind water resistant waterproof soked f...           0
7400  extremely confortable material soft cotton pou...           1

[7178 rows x 2 columns]


In [ ]:
# Putting reviews into the x axis, and sentiments into the y axis
x, y = (train_df['reviews'].values, train_df['sentiments'].values)

In [ ]:
# Tokenization + padding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tok = Tokenizer(lower = True)
tok.fit_on_texts(x)
x_sequence = tok.texts_to_sequences(x)
x_padding = pad_sequences(x_sequence, maxlen=32, padding='post')

In [ ]:
print(x_padding)

[[  12  274  207 ...    0    0    0]
 [   6   38   46 ...    0    0    0]
 [3359  131   50 ...  754 1179 2339]
 ...
 [1331  381  206 ...    0    0    0]
 [ 138  919  373 ...    0    0    0]
 [ 216  861  100 ...    0    0    0]]


In [ ]:
# split data into training and testing set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_padding, y, test_size = 0.25, random_state = 1)

In [ ]:
#building CNN model
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
vocabulary_size = len(tok.word_counts.keys())+1
max_words = 32
embedding_size = 200
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size = 32, epochs = 3)

Epoch 1/3
169/169 [==============================] - 43s 225ms/step - loss: 0.3517 - accuracy: 0.8679 - val_loss: 0.2676 - val_accuracy: 0.8891
Epoch 2/3
169/169 [==============================] - 26s 156ms/step - loss: 0.1678 - accuracy: 0.9422 - val_loss: 0.2679 - val_accuracy: 0.9019
Epoch 3/3
169/169 [==============================] - 26s 154ms/step - loss: 0.0900 - accuracy: 0.9760 - val_loss: 0.2994 - val_accuracy: 0.9014


In [401]:
accuracy = model.evaluate(x_test, y_test, verbose=1)
print("Test accuracy is: {:.2%}".format(accuracy[1]))

57/57 [==============================] - 4s 66ms/step - loss: 0.2994 - accuracy: 0.9014
Test accuracy is: 90.14%


In [ ]:
import numpy as np

# Tokenization + padding
x_test_set = test_df['reviews']
x_test_sequence = tok.texts_to_sequences(x_test_set)
x_test_padding = pad_sequences(x_test_sequence, maxlen = 32, padding = 'post')

# x_test_sequence = tok.texts_to_sequences(test_df['reviews'])
# x_test_padding = pad_sequences(x_test_sequence, maxlen=256)

# Make predictions
y_test_prediction_probablity = model.predict(x_test_padding)
y_test_prediction = np.round(y_test_prediction_probablity).astype(int)




225/225 [==============================] - 8s 35ms/step


In [ ]:
# Display the predictions
test_df['predicted_sentiment'] = y_test_prediction
print(test_df[['reviews', 'predicted_sentiment']])

                                                reviews  predicted_sentiment
0            bought belt daughter inlaw christmas loved                    1
1               size perfect color looked like web page                    1
2     fits feels good esp swim race highly recommend...                    1
3     socks absolutely best take pilates classes hot...                    1
4     thank much speedy delivery came time rehearsal...                    1
...                                                 ...                  ...
7396  bought shirts black medium wear daily basis di...                    0
7397  first thought scarf might good quality since c...                    1
7398  picky comes bras want something support comfor...                    1
7399  jacket wind water resistant waterproof soked f...                    0
7400  extremely confortable material soft cotton pou...                    1

[7178 rows x 2 columns]
